# 날씨 데이터 크롤링

출처: https://www.data.go.kr/  
ASOS 종관기상관측 데이터 활용

# URL 설정

# 시간별자료

In [66]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import datetime

In [2]:
key='13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D'
url="http://data.kma.go.kr/apiData/getData"


### api 설명서에 있는 활용가이드에서 요청/응답 메세지 예제를 확인한다

### urlparse, parse_qs를 통해 파라미터로 나눠준다

In [12]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=108&schListCnt=10&endDt=20200101&endHh=01&startHh=01&startDt=20100101&dataType=JSON")
par_dic = parse_qs(a.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'HR',
 'stnIds': '108',
 'schListCnt': '10',
 'endDt': '20200101',
 'endHh': '01',
 'startHh': '01',
 'startDt': '20100101',
 'dataType': 'JSON'}

In [135]:
def get_hourly_data(start_date, start_hour,end_date,end_hour):
    start_date=start_date
    start_hour=start_hour
    end_date=end_date
    end_hour=end_hour
    url='http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

    
    params={'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
     'numOfRows': '10',
     'pageNo': '1',
     'dataCd': 'ASOS',
     'dateCd': 'HR',
     'stnIds': '108',
     'schListCnt': '10',
     'endDt': end_date,
     'endHh': end_hour,
     'startHh': start_hour,
     'startDt': start_date,
     'dataType': 'JSON'
           }

    resp = requests.get(url, params=params)
    #result= json.loads(resp.text)

    return resp.text

In [136]:
data=get_hourly_data('20200605','08','20200605','09')
json.loads(data)

{'response': {'header': {'resultCode': '99',
   'resultMsg': '전날 자료까지 제공됩니다. 날짜 범위를 확인해주세요.'}}}

# 일별데이터 출력

In [56]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt=20100101&endDt=20100102&stnIds=108")
par_dic = parse_qs(url.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'DAY',
 'startDt': '20100101',
 'endDt': '20100102',
 'stnIds': '108'}

In [81]:
def get_daily_data(start_date):
    start_date=start_date 
    end_date=str(datetime.date.today()-datetime.timedelta(1)).replace('-','')
    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

    
    params={'serviceKey': unquote('13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D'),
            'numOfRows': '200',
            'pageNo': '1',
            'dataCd': 'ASOS',
            'dateCd': 'DAY',
            'startDt': start_date,
            'endDt': end_date,
            'stnIds': '108',
            'dataType':'JSON'
           }

    resp = requests.get(url, params=params)
    result= json.loads(resp.text)
    return result

In [60]:
def data_to_DataFrame(result):
    total_data=result['response']['body']['items']['item']
    
    result = {'날짜':[],
              '최저기온':[],
              '최고기온':[],
              '비':[],
              '강수량':[],
             }
    for data in total_data:
        result['날짜'].append(data['tm'])
        result['최저기온'].append(data['minTa'])
        result['최고기온'].append(data['maxTa'])
        if data['sumRn']>0:
            result['비'].append('O')
        else:
            result['비'].append('X')
        result['강수량'].append(data['sumRn'])
        
    df=pd.DataFrame(result)
    
    return df

In [96]:
def append_today(data):
    url = 'https://www.weatheri.co.kr/forecast/forecast01.php'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, from_encoding='utf-8')
    attrs={'width':'615',
      'border':'0',
      'cellpadding':'1',
      'cellspacing':'1',
       'bgcolor':'#BCBFC2'
      }
    inner_table = soup.find('table',attrs=attrs)
    date = inner_table.select('td b')[0].text
    date = '2020-{0}-{1}'.format(date.split()[0][:2],date.split()[1][:2])
    high = inner_table.select('table tr td b font')[0].text
    high = high.replace('˚C','')
    low = inner_table.select('table tr td b font')[0].text
    low = low.replace('˚C','')
    sum_rain=inner_table.select('table tr td font')[-1].text
    if sum_rain !='- mm':
        rain='O'
        sum_rain = sum_rain.split()[0]
        
    else:
        rain='X'
        sum_rain = 0
        
    new_data=pd.DataFrame({'날짜':[date],
             '최저기온':[low],
             '최고기온':[high],
             '비':[rain],
             '강수량':[sum_rain]})
    print(sum_rain)
    data = data.append(new_data,ignore_index=True)
    print(data)
    return data

In [97]:
result=get_daily_data('20200501')
output=data_to_DataFrame(result)
d=append_today(output)

21.1
            날짜  최저기온  최고기온  비   강수량
0   2020-05-01  16.4  26.2  X     0
1   2020-05-02    18  23.9  X     0
2   2020-05-03    17  27.4  X     0
3   2020-05-04  14.8  25.3  X     0
4   2020-05-05  13.1  19.3  X     0
5   2020-05-06  11.1  27.6  X     0
6   2020-05-07  14.9  26.5  X     0
7   2020-05-08  14.3  27.3  O   2.3
8   2020-05-09  12.7  16.8  O  24.4
9   2020-05-10  12.8  16.6  O   1.3
10  2020-05-11  12.4    22  O   3.4
11  2020-05-12  10.5  18.9  O   0.7
12  2020-05-13  10.2  21.8  X     0
13  2020-05-14  12.4  25.4  X     0
14  2020-05-15  14.1  20.2  O    12
15  2020-05-16    15  20.3  O   2.5
16  2020-05-17  14.6  24.9  X     0
17  2020-05-18  13.2  26.8  O  29.8
18  2020-05-19  10.1  17.1  O    19
19  2020-05-20   9.2  20.1  X     0
20  2020-05-21  10.9  22.9  X     0
21  2020-05-22  14.8  22.7  X     0
22  2020-05-23  16.6    25  X     0
23  2020-05-24  14.3  20.2  O  14.5
24  2020-05-25  13.7  21.3  X     0
25  2020-05-26  12.7  22.9  O   1.5
26  2020-05-27  11.8  2

In [65]:
d.to_excel('weather_crawling.xlsx')